# Statistical Inference and Linear Regression

<hr>

Sergei Yu. Papulin (papulin.study@yandex.ru)

### Contents

1. [Loading Initial Data](#Loading-Initial-Data)
2. [Simple Regression](#Simple-Regression)
    - Estimating Parameters using OLS
    - Estimating Standard Errors
    - Confidence and Prediction Intervals
    - Hypothesis Tests
3. [Multiple Linear Regression](#Multiple-Linear-Regression)
    - Estimating Parameters using OLS
    - Estimating Standard Errors
    - Confidence and Prediction Intervals
    - Hypothesis Tests
    - Using Statsmodels Library
4. [References](#References)
 

Import modules and functions that will be used later

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats
from numpy.linalg import inv

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

In [ ]:
import sys
sys.path.insert(1, "../lib/")
import plot_stats as plot_stats

## Loading Initial Data

In [ ]:
FILE_PATH = "../data/Advertising.csv"

In [ ]:
df = pd.read_csv(FILE_PATH, sep=",", index_col=0)
df.head(5)

## Simple Regression

In [ ]:
# Scatter plot

x = df["TV"].to_numpy()
y = df["sales"].to_numpy()

plt.figure("1", figsize=[10, 6])

plt.subplot(1,1,1)
plt.title("TV advertisement - Sales")
plt.scatter(x, y, color="slategrey")
plt.xlabel("TV")
plt.ylabel("sales")
plt.grid(True)

plt.show()

### Estimating Parameters using OLS

$$\hat{\theta}=\left(\begin{matrix}{\hat{\theta}}_0\\\begin{matrix}{\hat{\theta}}_1\\\vdots\\\end{matrix}\\{\hat{\theta}}_p\\\end{matrix}\right)=\left(X^TX\right)^{-1}X^Ty$$

In [ ]:
X = np.array(np.c_[np.ones(x.size), x])
w, residuals, rank, s = np.linalg.lstsq(X, y.reshape(y.size, 1), rcond=None)
w0, w1 = w[0,0], w[1,0]
w0, w1 

In [ ]:
y_pred = w0 + w1*x
y_pred[:5]

In [ ]:
# Scatter plot

plt.figure("1", figsize=[12, 4])

plt.subplot(1,2,1)
plt.title("Linear Regression")
plt.scatter(x, y, color="slategrey")
plt.xlabel("TV")
plt.ylabel("sales")
plt.grid(True)


# Linear Functions

x_min_max_indx = np.array([x.argmin(), x.argmax()])
plt.plot(x[x_min_max_indx], y_pred[x_min_max_indx], "-r", label="$f_1(x)=0.9*x+0.3$")


plt.subplot(1,2,2)
plt.title("Residual Plot")
plt.scatter(x, y-y_pred, color="slategrey")
plt.xlabel("TV")
plt.ylabel("sales")
plt.grid(True)


plt.show()

### Estimating Standard Errors

Mean:

In [ ]:
x_mean = x.mean()
x_mean

Resuduals:

In [ ]:
e = y - y_pred
e[:5]

Residual standard error:

$$s^2=\frac{1}{n-2}\sum_{i=1}^{n}\left(y_i-{\hat{y}}_i\right)^2$$

In [ ]:
s = e.std(ddof=2)
s

Standard error of the estimate $\theta_0$:

$$SE({\hat{\theta}}_0)=s\sqrt{\frac{1}{n}+\frac{{\bar{x}}^2}{\sum_{i=1}^{n}\left(x_i-\bar{x}\right)^2}}$$

In [ ]:
def SE_w0(x, x_mean, s):
    n = x.size
    return (s**2*(1/n + x_mean**2/((x - x_mean)**2).sum()))**0.5

In [ ]:
se_w0 = SE_w0(x, x_mean, s)
se_w0

Standard error of the estimate $\theta_1$:

$$SE({\hat{\theta}}_1)=\frac{s}{\sqrt{\sum_{i=1}^{n}\left(x_i-\bar{x}\right)^2}}$$

In [ ]:
def SE_w1(x, x_mean, s):
    n = x.size
    return (s**2/((x - x_mean)**2).sum())**0.5

In [ ]:
se_w1 = SE_w1(x, x_mean, s)
se_w1

### Confidence and Prediction Intervals

#### Confidence Intervals for parameters

In [ ]:
ALPHA = 0.05

In [ ]:
dof = y.size - 2

In [ ]:
t_alpha = stats.t.ppf(1 - ALPHA/2, dof) 
t_alpha

$${\hat{\theta}}_0\pm  t_{1-\alpha/2,n-2}SE({\hat{\theta}}_0)$$

In [ ]:
lower_ci_w0, upper_ci_w0 = w0 - t_alpha * se_w0, w0 + t_alpha * se_w0
lower_ci_w0, upper_ci_w0

$${\hat{\theta}}_1\pm  t_{1-\alpha/2,n-2}SE({\hat{\theta}}_1)$$

In [ ]:
lower_ci_w1, upper_ci_w1 = w1 - t_alpha * se_w1, w1 + t_alpha * se_w1
lower_ci_w1, upper_ci_w1

#### Confidence Interval

$$SE\left({\hat{y}}_i\right)=\sqrt{s^2\left(\frac{1}{n}+\frac{\left(x_i-\bar{x}\right)^2}{\sum_{i=1}^{n}\left(x_i-\bar{x}\right)^2}\right)}$$

In [ ]:
def SE_y(x, x_mean, s):
    n = x.size
    return (s**2*(1/n + (x - x_mean)**2/((x - x_mean)**2).sum()))**0.5

In [ ]:
se_y = SE_y(x, x_mean, s)
se_y[:5]

$${\hat{y}}_i\pm t_{1-\alpha/2,n-2}SE\left({\hat{y}}_i\right)$$

In [ ]:
lower_ci_y, upper_ci_y = y_pred - t_alpha*se_y, y_pred + t_alpha*se_y
lower_ci_y[:5], upper_ci_y[:5]

In [ ]:
# Scatter plot

plt.figure("1", figsize=[10, 6])

plt.subplot(1,1,1)

plt.title("Confidence Interval")

plt.scatter(x, y, color="slategrey")
plt.xlabel("TV")
plt.ylabel("sales")
plt.grid(True)


# Linear Functions

x_min_max_indx = np.array([x.argmin(), x.argmax()])

plt.plot(x[x_min_max_indx], y_pred[x_min_max_indx], "-", 
         color="MidnightBlue", 
         linewidth=2, 
         label="$f_{pred}(x)$")


# CI

ci_y_stack = np.stack([x, upper_ci_y, lower_ci_y], axis=1)
ci_y_sorted = ci_y_stack[ci_y_stack[:,0].argsort()]

plt.fill_between(ci_y_sorted[:,0], ci_y_sorted[:,1], ci_y_sorted[:,2], 
                 facecolor="lightblue", alpha=0.5, 
                 linewidth=0.5, edgecolor="blue", label="CI")


plt.legend()

#plt.tight_layout()
plt.autoscale(True, tight=True)

plt.show()

#### Prediction Interval

$$SE\left(e\right)=\sqrt{s^2+{SE\left({\hat{y}}_i\right)}^2}$$

In [ ]:
def SE_e(x, x_mean, s):
    n = x.size
    return (s**2*(1 + 1/n + (x - x_mean)**2/((x - x_mean)**2).sum()))**0.5

In [ ]:
se_e = SE_e(x, x_mean, s)
se_e[:5]

$${\hat{y}}_\ast\pm t_{1-\alpha/2,n-2}\sqrt{s^2\left(1+\frac{1}{n}+\frac{\left(x_\ast-\bar{x}\right)^2}{\sum_{i=1}^{n}\left(x_i-\bar{x}\right)^2}\right)}$$

In [ ]:
lower_ci_e, upper_ci_e = y_pred - t_alpha*se_e, y_pred + t_alpha*se_e
lower_ci_e[:5], upper_ci_e[:5]

In [ ]:
# Scatter plot

plt.figure("1", figsize=[10, 6])

plt.subplot(1,1,1)

plt.title("Prediction Interval")

plt.scatter(x, y, color="slategrey")
plt.xlabel("TV")
plt.ylabel("sales")
plt.grid(True)


# Linear Functions

x_min_max_indx = np.array([x.argmin(), x.argmax()])

plt.plot(x[x_min_max_indx], y_pred[x_min_max_indx], "-", 
         color="MidnightBlue", linewidth=2, 
         label="$f_{pred}(x)$")



# PI

ci_e_stack = np.stack([x, upper_ci_e, lower_ci_e], axis=1)
ci_e_sorted = ci_e_stack[ci_e_stack[:,0].argsort()]

plt.fill_between(ci_e_sorted[:,0], ci_e_sorted[:,1], ci_e_sorted[:,2], 
                 facecolor="lightgreen", alpha=0.5, 
                 linewidth=0.5, edgecolor="green", 
                 label="PI")


# CI

ci_y_stack = np.stack([x, upper_ci_y, lower_ci_y], axis=1)
ci_y_sorted = ci_y_stack[ci_y_stack[:,0].argsort()]

plt.fill_between(ci_y_sorted[:,0], ci_y_sorted[:,1], ci_y_sorted[:,2], 
                 facecolor="lightblue", alpha=0.5, 
                 linewidth=0.5, edgecolor="blue",
                 label="CI")

plt.legend()

plt.autoscale(True, tight=True)

plt.show()

### Hypothesis Tests

#### t-Test

$$H_0: \theta_0=0$$

$$H_a: \theta_0\neq0$$

In [ ]:
def calc_t(x, mu, se):
    return (x - mu) / se

In [ ]:
t_w0 = calc_t(w0, 0, se_w0)
t_w0

In [ ]:
pvalue_t_w0 = 2 * stats.t.cdf(-abs(t_w0), dof)
pvalue_t_w0

In [ ]:
plt.figure(1, figsize=[8,4])

plt.subplot(1,1,1)
plot_stats.plot_two_tailed_pvalue_for_tdistribution(t_w0, dof, ALPHA, xlim=(-5, 20))

plt.show()

$$H_0: \theta_1=0$$

$$H_a: \theta_1\neq0$$

In [ ]:
t_w1 = calc_t(w1, 0, se_w1)
t_w1

In [ ]:
pvalue_t_w1 = 2 * stats.t.cdf(-abs(t_w1),dof)
pvalue_t_w1

In [ ]:
plt.figure(2, figsize=[8,4])

plt.subplot(1,1,1)
plot_stats.plot_two_tailed_pvalue_for_tdistribution(t_w1, dof, ALPHA, xlim=(-5, 20))

plt.show()

## Multiple Linear Regression

In [ ]:
# Scatter plot

feature_clmns = ["TV", "radio", "newspaper"]
response_clmn = "sales"

X = df[feature_clmns].to_numpy()
y = df[response_clmn].to_numpy()

plt.figure("1", figsize=[14, 4])

for i in range(len(feature_clmns)):
    plt.subplot(1, len(feature_clmns), i+1)
    plt.title("{} advertisement - Sales".format(feature_clmns[i]))
    plt.scatter(X[:,i], y, color="slategrey")
    plt.xlabel(feature_clmns[i])
    plt.ylabel("sales")
    plt.grid(True)

plt.show()

### Estimating Parameters using OLS

In [ ]:
X_ext = np.c_[np.ones(y.size), X]
w, residuals, rank, s = np.linalg.lstsq(X_ext, y.reshape(y.size, 1), rcond=None)
w = w.flatten()
w

In [ ]:
num_params = w.size
num_params

In [ ]:
y_pred = X_ext.dot(w).flatten()
y_pred[:5]

In [ ]:
y_pred = w[0] + w[1]*X[:,0] + w[2]*X[:,1] + w[3]*X[:,2]
y_pred[:5]

### Estimating Standard Errors

In [ ]:
e = y - y_pred
e[:5]

In [ ]:
s = e.std(ddof=num_params)
s

$$C = s^{2}(X^TX)^{-1}$$

In [ ]:
inv_XTX = inv(X_ext.T.dot(X_ext))

In [ ]:
C = s**2*inv_XTX
C

$$SE(\hat{\theta_i}) = \sqrt{C_{ii}}$$

In [ ]:
se_w = np.sqrt(C.diagonal())
se_w

### Confidence and Prediction Intervals

#### Confidence Intervals for parameters

In [ ]:
dof = y.size - num_params

In [ ]:
t_alpha = stats.t.ppf(1 - ALPHA/2, dof) 
t_alpha

In [ ]:
lower_ci_w, upper_ci_w = w - t_alpha * se_w, w + t_alpha * se_w
lower_ci_w, upper_ci_w

#### Confidence Interval

In [ ]:
SE_y = lambda x: ((s**2*x.T.dot(inv_XTX).dot(x))**0.5).flatten()[0]
CI_y =  lambda y_pred, se, t_alpha: (y_pred - t_alpha*se, y_pred + t_alpha*se)

In [ ]:
x = X_ext[2,:]
x

In [ ]:
se_y = SE_y(x.reshape(num_params,1))
se_y

In [ ]:
lower_ci_y, upper_ci_y = CI_y(y_pred[0], se_y, t_alpha)
lower_ci_y, upper_ci_y

#### Prediction Interval

In [ ]:
SE_e = lambda x: ((s**2*(1+x.T.dot(inv_XTX).dot(x)))**0.5).flatten()[0]
CI_e =  lambda y_pred, se, t_alpha: (y_pred - t_alpha*se, y_pred + t_alpha*se)

In [ ]:
x = X_ext[2,:]
x

In [ ]:
se_e = SE_e(x.reshape(num_params,1))
se_e

In [ ]:
lower_ci_e, upper_ci_e = CI_e(y_pred[0], se_e, t_alpha)
lower_ci_e, upper_ci_e

### Hypothesis Tests

#### t-Test

In [ ]:
t_w = calc_t(w, 0, se_w)
t_w

In [ ]:
pvalue_t = 2 * stats.t.cdf(-abs(t_w), dof)
pvalue_t

In [ ]:
plt.figure(4, figsize=[8,4])

plt.subplot(1,1,1)
plot_stats.plot_two_tailed_pvalue_for_tdistribution(t_w[3], dof, ALPHA, xlim=(-4, 4))

plt.show()

### Using Statsmodels Library

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.OLS(y, X_ext)
results = model.fit()

In [ ]:
results.params

In [ ]:
results.summary()

#### Formula

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
formula = "sales ~ TV + radio + newspaper"
model = ols(formula, df)
results = model.fit()
results.summary()

#### Hypothesis Tests

In [ ]:
hypotheses = "Intercept = 0, TV = 0, radio = 0, newspaper = 0"
t_test = results.t_test(hypotheses)
print(t_test)

## References

1. Chapter 3. Linear Regression // [An Introduction to Statistical Learning](http://faculty.marshall.usc.edu/gareth-james/ISL/) by Gareth James, Daniela Witten, Trevor Hastie, Robert Tibshir
2. [Chapter 3: Simple Linear Regression Analysis](http://reliawiki.org/index.php/Simple_Linear_Regression_Analysis)
2. [Chapter 4: Multiple Linear Regression Analysis](http://reliawiki.org/index.php/Multiple_Linear_Regression_Analysis)